<a href="https://colab.research.google.com/github/christianzambra/productmanagerslife/blob/main/contar_e_classificar_palavras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyPDF2 unidecode



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 10.1 MB/s eta 0:00:00


In [74]:
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 24.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [75]:
import PyPDF2
from collections import Counter
import re
from unidecode import unidecode
import spacy

In [76]:
# from spacy.lang.pt.examples import sentences

# nlp = spacy.load("pt_core_news_sm")
# doc = nlp(sentences[0])
# print(doc.text)
# for token in doc:
#     print(token.text, token.pos_, token.dep_)

Apple está querendo comprar uma startup do Reino Unido por 100 milhões de dólares
Apple PROPN nsubj
está AUX aux
querendo VERB ROOT
comprar VERB xcomp
uma DET det
startup NOUN obj
do ADP case
Reino PROPN nmod
Unido PROPN flat:name
por ADP case
100 NUM obl
milhões NUM flat
de ADP case
dólares NOUN nmod


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [50]:
def read_pdf_count_words(file_path, exclude_words=[]):
    # Open the PDF file in binary mode
    with open(file_path, 'rb') as file:
        # Create a PDF reader object
        pdf_reader = PyPDF2.PdfReader(file)

        # Initialize an empty string to store the extracted text
        text = ''

        # Iterate through each page and extract text
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()

    # Normalize the text to remove accents and convert to lowercase
    normalized_text = unidecode(text.lower())

    # Use regular expression to extract words (ignoring non-alphanumeric characters)
    words = re.findall(r'\b\w+\b', normalized_text)

    # Exclude specified words
    filtered_words = [word for word in words if word not in exclude_words]

    # Count the occurrences of each word
    word_counts = Counter(filtered_words)

    # Get the top 5 words
    top_words = word_counts.most_common(20)

    # Display the results
    print(f"Total words: {len(words)}")
    print("Top 5 words:")
    for word, count in top_words:
        print(f"{word}: {count}")

In [85]:
def classify_and_list_top_n(file_path, exclude_words=[], top_n=5):
    # Open the PDF file in binary mode
    with open(file_path, 'rb') as file:
        # Create a PDF reader object
        pdf_reader = PyPDF2.PdfReader(file)

        # Initialize an empty string to store the extracted text
        text = ''

        # Iterate through each page and extract text
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()

    # Normalize the text to remove accents and convert to lowercase
    normalized_text = unidecode(text.lower())

    # Use regular expression to extract words (ignoring non-alphanumeric characters)
    words = re.findall(r'\b\w+\b', normalized_text)

    # Exclude specified words
    filtered_words = [word for word in words if word not in exclude_words]

    # Join the filtered words into a string
    text_for_spacy = ' '.join(filtered_words)

    # Load spaCy with the Portuguese model
    nlp = spacy.load('pt_core_news_sm')

    # Process the text with spaCy
    doc = nlp(text_for_spacy)

    # Initialize counters for nouns and verbs
    noun_counts = Counter()
    verb_counts = Counter()

    propn_counts = Counter()
    num_counts = Counter()
    adp_counts = Counter()
    aux_counts = Counter()
    det_counts = Counter()

    # Classify words into nouns and verbs
    for token in doc:
        if token.pos_ == 'NOUN':
            noun_counts[token.text] += 1
        elif token.pos_ == 'VERB':
            verb_counts[token.text] += 1

        elif token.pos_ == 'PROPN':
            propn_counts[token.text] += 1
        elif token.pos_ == 'NUM':
            num_counts[token.text] += 1
        elif token.pos_ == 'ADP':
            adp_counts[token.text] += 1
        elif token.pos_ == 'AUX':
            aux_counts[token.text] += 1
        elif token.pos_ == 'DET':
            det_counts[token.text] += 1




    # Get the top N most used nouns and verbs
    top_n_nouns = [word for word, count in noun_counts.most_common(top_n)]
    top_n_verbs = [word for word, count in verb_counts.most_common(top_n)]

    top_n_propn = [word for word, count in propn_counts.most_common(top_n)]
    top_n_num = [word for word, count in num_counts.most_common(top_n)]
    top_n_adp = [word for word, count in adp_counts.most_common(top_n)]
    top_n_aux = [word for word, count in aux_counts.most_common(top_n)]
    top_n_det = [word for word, count in det_counts.most_common(top_n)]

    # Display the results
    print(f"In the text, the {top_n} most used nouns are: {', '.join(top_n_nouns)}")
    print(f"The {top_n} most used verbs are: {', '.join(top_n_verbs)}")

    print(f"The {top_n} most used proper name are: {', '.join(top_n_propn)}")
    print(f"The {top_n} most used numbers are: {', '.join(top_n_num)}")
    print(f"The {top_n} most used adp are: {', '.join(top_n_adp)}")
    print(f"The {top_n} most used aux are: {', '.join(top_n_aux)}")
    print(f"The {top_n} most used det are: {', '.join(top_n_det)}")

In [51]:
import os

# List files in the specified directory
directory_path = '/content/drive/MyDrive/teste_conta_palavras/'
files = os.listdir(directory_path)
print(files)

['DomCasmurro.pdf']


In [83]:
if __name__ == "__main__":
    # Specify the correct filename in the path
    pdf_file_path = '/content/drive/MyDrive/teste_conta_palavras/DomCasmurro.pdf'
    exclude_words = [
        "e",
        "a",
        "que",
        "de",
        "o",
        "nao",
        "me",
        "se",
        "um",
        "os",
        "da",
        "as",
        "mas",
        "do",
        "era",
        "para",
        "com",
        "eu",
        "lhe",
        "em",
        "uma",
        "como",
        "ao",
        "por",
        "minha",
        "no",
        "mais",
        "na",
        "ou",
        "nem",
        "ele",
        "mae",
        "foi",
        "quando",
        "dias",
        "tambem",
        "tudo",
        "ela",
        "dos",
        "disse",
        "so",
        "ser",
        "meu",
        "nos",
        "ja",
        "esta",
        "mim",
        "assim",
        "ha",
        "muito",
        "nead",
        "unama"

        ]




    read_pdf_count_words(pdf_file_path, exclude_words)

Total words: 67467
Top 5 words:
capitu: 338
casa: 170
capitulo: 164
olhos: 164
jose: 159
depois: 151
vez: 146
agora: 146
das: 145
br: 141
la: 141
outra: 138
ainda: 138
sem: 137
voce: 136
nada: 133
tinha: 130
lo: 127
tao: 122
mesmo: 120


In [87]:
if __name__ == "__main__":
    # Specify the correct filename in the path
    pdf_file_path = '/content/drive/MyDrive/teste_conta_palavras/DomCasmurro.pdf'

    # Specify the list of words to exclude from the count
    #exclude_words = ["a", "e", "o", "lhe"]  # Add more words as needed
    exclude_words = ["unad","uama","br"]
    # Specify the number of most used words to display
    top_n = 5

    classify_and_list_top_n(pdf_file_path, exclude_words, top_n)

In the text, the 5 most used nouns are: mae, capitu, casa, olhos, vez
The 5 most used verbs are: disse, tambem, nao, tinha, dizer
The 5 most used proper name are: ja, dias, la, capitulo, nao
The 5 most used numbers are: um, duas, uma, lo, www
The 5 most used adp are: de, da, do, com, a
The 5 most used aux are: era, foi, ser, nao, eram
The 5 most used det are: a, o, um, os, as
